In [1]:
import pandas as pd
import json
import time

#Carregar os Dados
try:
    df_reviews = pd.read_csv('olist_order_reviews_dataset.csv')
    # Filtra apenas reviews que tenham comentários (não nulos) e pega uma amostra de 10
    df_sample = df_reviews[df_reviews['review_comment_message'].notnull()].head(10).copy()
    print(f"Amostra carregada: {len(df_sample)} reviews.")
except FileNotFoundError:
    print("ERRO: Faça upload do arquivo 'olist_order_reviews_dataset.csv' no Colab!")

def simular_llm(comentario):

    comentario = comentario.lower()
    sentimento = "Neutro"
    categoria = "Geral"

    if any(x in comentario for x in ['bom', 'otimo', 'gostei', 'excelente', 'rápido']):
        sentimento = "Positivo"
    elif any(x in comentario for x in ['ruim', 'atraso', 'quebrado', 'não chegou', 'péssimo']):
        sentimento = "Negativo"

    if "entrega" in comentario or "prazo" in comentario or "correio" in comentario:
        categoria = "Logística"
    elif "produto" in comentario or "qualidade" in comentario:
        categoria = "Produto"

    # Estrutura JSON
    json_output = {
        "sentiment": sentimento,
        "category": categoria,
        "features": {
            "keywords": comentario.split()[:3], # Pega as 3 primeiras palavras como tags
            "urgent_action_needed": "Sim" if sentimento == "Negativo" else "Não"
        }
    }
    return json.dumps(json_output)

# Aplicando a IA nos dados
print("Processando reviews com GenAI (Simulado)...")
df_sample['ai_features_json'] = df_sample['review_comment_message'].apply(simular_llm)


df_features = pd.json_normalize(df_sample['ai_features_json'].apply(json.loads))

# Juntando com o ID original
df_final = pd.concat([df_sample.reset_index(drop=True), df_features], axis=1)

# Selecionando colunas finais para o relatório
colunas_finais = ['review_id', 'review_comment_message', 'sentiment', 'category', 'ai_features_json']
df_export = df_final[colunas_finais]

#Salvar
df_export.to_csv("REVIEWS_ENRICHED_GENAI.csv", index=False)
print("\nProcessamento Concluído!")
print("Exemplo de Saída:")
print(df_export[['review_comment_message', 'sentiment', 'category']].head(3))

Amostra carregada: 10 reviews.
Processando reviews com GenAI (Simulado)...

Processamento Concluído!
Exemplo de Saída:
                              review_comment_message sentiment   category
0              Recebi bem antes do prazo estipulado.    Neutro  Logística
1  Parabéns lojas lannister adorei comprar pela I...    Neutro      Geral
2  aparelho eficiente. no site a marca do aparelh...    Neutro      Geral
